# tf.keras.losses

In the supervised learning, we calculate the gradient from the loss function, which usually contains the estimation error and regularization.

In keras model, the regularization in loss function is defined in the layer. For example, the oprtion of kernel_regularizer and bias regularizer in Dense layer. The estimation error is defined during the compilation model. The mean squared error is frequently used for regression problem. The binary_crossentropy is used for binary classification problem, the categorical_crossentropy is for multi-classification problme. 

Sometime, we also need to define specific error function with y_true and y_pred as the input and a scalar number as the output.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, losses, regularizers, constraints

## 1. loss function 

In [2]:
tf.keras.backend.clear_session()

model = models.Sequential()
model.add(layers.Dense(64, input_dim=64,
                kernel_regularizer=regularizers.l2(0.01), 
                activity_regularizer=regularizers.l1(0.01),
                kernel_constraint = constraints.MaxNorm(max_value=2, axis=0))) 
model.add(layers.Dense(10,
        kernel_regularizer=regularizers.l1_l2(0.01,0.01),activation = "sigmoid"))
model.compile(optimizer = "rmsprop",
        loss = "sparse_categorical_crossentropy", metrics = ["AUC"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 4,810
Trainable params: 4,810
Non-trainable params: 0
_________________________________________________________________


## 2. build-in loss function

    1. mean_squared_error: "mse"
    2. mean_absolute_error: "mae"
    3. mean_absolute_percentage_error: "mape"
    4. Huber
    5. binary_crossentropy
    6. categorical_crossentropy
    7. sparse_categorical_crossentropy
    8. hinge: SVM
    9. kld: measure the simularity of 2 distributions (EM)
    10. cosine_similarity

## 3. custom loss function

The custom loss function can be realized by a function or a class inherited from tf.keras.losses.Loss. For the second realization, call function should be overrided to define the calculation.

We realize the focal loss as an example.

In [3]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        loss = -tf.sum(alpha * tf.pow(1. - pt_1, gamma) * tf.log(1e-07+pt_1)) \
           -tf.sum((1-alpha) * tf.pow( pt_0, gamma) * tf.log(1. - pt_0 + 1e-07))
        return loss
    return focal_loss_fixed

In [4]:
class FocalLoss(losses.Loss):

    def __init__(self,gamma=2.0,alpha=0.25):
        self.gamma = gamma
        self.alpha = alpha

    def call(self,y_true,y_pred):

        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        loss = -tf.sum(self.alpha * tf.pow(1. - pt_1, self.gamma) * tf.log(1e-07+pt_1)) \
           -tf.sum((1-self.alpha) * tf.pow( pt_0, self.gamma) * tf.log(1. - pt_0 + 1e-07))
        return loss